In [1]:

import os
import torch
import torch_neuronx
from transformers import AutoTokenizer, AutoModelForSequenceClassification


/home/ec2-user/aws_neuron_venv_pytorch_p37/lib64/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Build tokenizer and model
# model_id = "yiyanghkust/finbert-tone"
model_id = "cardiffnlp/twitter-roberta-base-sentiment"
# model_id = "thusken/nb-bert-large-user-needs"


tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSequenceClassification.from_pretrained(model_id, torchscript=True)


In [3]:
# create dummy input for max length 128
def generate_sample_inputs(tokenizer, *payload, sequence_length=128):
  embeddings = tokenizer(*payload, max_length=sequence_length, padding="max_length",return_tensors="pt")
  return tuple(embeddings.values())


payload = "i am happy that my stock grew5%."
neuron_inputs = generate_sample_inputs(tokenizer,payload)

In [4]:
# Run the original PyTorch model on examples
with torch.inference_mode():
  reference_logits = model(*neuron_inputs)[0]


In [5]:
import os 
# use only one neuron core
os.environ["NEURON_RT_NUM_CORES"] = "1"

# Run precompiled TorchScript that is optimized by AWS Neuron Inf2
neuron_model = torch_neuronx.trace(model, neuron_inputs)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


12/08/2022 03:24:21 PM WARNING 112653 [py.warnings]: /home/ec2-user/aws_neuron_venv_pytorch_p37/bin/neuronx-cc:8: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  sys.exit(main())

12/08/2022 03:24:41 PM WARNING 112653 [py.warnings]: /home/ec2-user/aws_neuron_venv_pytorch_p37/bin/neuronx-cc:8: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  sys.exit(main())

12/08/2022 03:24:44 PM WARNING 112653 [WalrusDriver]: 100% PSUM demand before spilling
12/08/2022 03:24:44 PM WARNING 112653 [WalrusDriver]: spilling from PSUM cost about 3888 cycles
12/08/2022 03:24:44 PM WARNING 112653 [WalrusDriver]: 100% PSUM utilization after allocatio

In [9]:
# Verify the TorchScript works on both example inputs
neuron_logits = neuron_model(*neuron_inputs)

In [10]:
print('Reference Logits:    ', reference_logits.detach().numpy())
print('Neuron Logits:       ', neuron_logits.detach().numpy())

Reference Logits:     [[-2.8316336 -0.3366956  3.8156364]]
Neuron Logits:        [[-2.8326252  -0.33097273  3.8116004 ]]


In [11]:
scores = torch.nn.Softmax(dim=-1)(neuron_logits)[0]

[{"label": model.config.id2label[i], "score": score.item()} for i, score in enumerate(scores)]

[{'label': 'LABEL_0', 'score': 0.001279529184103012},
 {'label': 'LABEL_1', 'score': 0.015613634139299393},
 {'label': 'LABEL_2', 'score': 0.9831068515777588}]